In [7]:
import pymysql
conn = pymysql.connect(host='192.168.1.184',
                      user='root', port=3306, password='acorn1234', db='acornschool',
                      charset='utf8mb4',use_unicode=True,
                      cursorclass=pymysql.cursors.DictCursor)

cur = conn.cursor()

#테이블 생성
cur.execute(""" CREATE TABLE movie(bunho INT NOT NULL AUTO_INCREMENT,
            title VARCHAR(50) NOT NULL, genre VARCHAR(20), date DATE, runtime INT, country VARCHAR(20),
            grade INT, count INT,
            PRIMARY KEY (bunho)) ENGINE=lnnoDB AUTO_INCREMENT=1""")
print(cur.description)
conn.close()

None


C:\Users\ICT01_15\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1286, "Unknown storage engine 'lnnoDB'")
  self._do_get_result()
C:\Users\ICT01_15\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1266, "Using storage engine InnoDB for table 'movie'")
  self._do_get_result()


In [13]:
conn = pymysql.connect(host='192.168.1.184',
                      user='root', port=3306, password='acorn1234', db='acornschool',
                      charset='utf8mb4',use_unicode=True,
                      cursorclass=pymysql.cursors.DictCursor)

cur = conn.cursor()

sql = """INSERT INTO movie(title, genre, date, runtime, country, grade, count)
        VALUES(%s, %s, %s, %s, %s, %s)"""
cur.execute(sql, ('천문: 하늘에 묻는다','2019-12-26','132','한국','12','1336430'))
conn.commit()
conn.close

<bound method Connection.close of <pymysql.connections.Connection object at 0x000002880D4A3348>>

In [39]:
class Movie:
    def __init__(self):
        self.bunho = 0
        self.title = ''
        self.genre = ''
        self.date = ''
        self.runtime = 0
        self.country = ''
        self.grade = ''
        self.count = 0
        
    def inputData(self):
        self.title = input("영화 제목 : ")
        self.genre = input("영화 장르 : ")
        self.date = input("개봉(예정)일: ")
        self.runtime = int(input("상영 시간: "))
        self.country = input("개봉국: ")
        self.grade = input("관람 등급 : ")
        self.count = int(input("관람객 수 : "))
                
    def __str__(self):
        return '%5s %5s %5s %6s %6s %6s %7s %7s ' % (self.bunho, self.title, self.genre, self.date, self.runtime, self.country,
                                                   self.grade, self.count)
    
    def __cmp__(self,other): # compare
        return self.title == other.title
    
    def __call__(self,other): # 클래스를 함수처럼 사용하고자 할 때 오버라이딩
        mv = []
        
        for movie in other:
            stp = Movie()
            
            stp.bunho = movie[0]
            stp.title = movie[1]
            stp.genre = movie[2]
            stp.date = movie[3]
            stp.runtime = movie[4]
            stp.country = movie[5]
            stp.grade = movie[6]
            stp.count = movie[7]
            mv.append(stp)
        return mv

In [40]:
import pymysql

class maria: # 마리아 디비 클래스
    
    schoolname = "영화 목록표"
    
    def __init__(self):
        self.conn = pymysql.connect(host='192.168.1.184', port=3306, user='root',
                           password='acorn1234',db='acornschool', charset='utf8',
                                    autocommit=True)
        cor = self.conn.cursor()
        cor.execute("SHOW TABLES")
        tables = cor.fetchall()
        c = []
        for i in tables:
            if  i == ('movie',):
                c.append(i)
        if len(c) != 0:
            movie_table = "SELECT * FROM movie"
            cor.execute(movie_table)
            print('테이블이 있습니다. ')
        else:
            cor.execute("""CREATE TABLE movie (title VARCHAR(50), genre VARCHAR(20), date DATE,  runtime INTEGER(11), country VARCHAR(20),
                        grade VARCHAR(20), count INTEGER(11))""")
            print('movie 테이블을 생성하였습니다. ')
    
    def __del__(self): # 시스템 종료(?)
        self.conn.close()

    def selectdb(self ): # 출력 프로시저 호출 및 실행
        cur = self.conn.cursor()
        cur.callproc("movie_select")
        if (cur.rowcount):
            stu = list(cur.fetchall())
        else :
            stu = 0;
        cur.close()
        return stu
   
    def insertdb(self, title, genre, date, runtime, country, grade, count): # 삽입 프로시저 호출 및 실행
        cur = self.conn.cursor()
        args = ( title, genre, date, runtime, country, grade, count )
        cur.callproc("movie_insert",args)
        cur.close()
       
    def updatedb(self, in_title, up_title): # 업데이트 프로시저 호출 및 실행
        cur = self.conn.cursor()
        args = (in_title, up_title, 0)
        cur.callproc("movie_update", args)
        cur.execute('SELECT @_movie_update_2') # movie_update 프로시저의 2번 변수(out_res) 출력
        result = cur.fetchone()
        if result==2 :
            print("수정할 영화 제목이 없습니다.")
        else :
            print("제목을 성공적으로 수정하였습니다. ")
        cur.close()
        
        return result 
   
    def deletedb(self,in_title):
        cur = self.conn.cursor()
        args = (in_title, 0)
        cur.callproc("movie_delete", args)
        cur.execute('SELECT @_movie_delete_1') # movie_delete 프로시저의 1번 변수(out_res) 출력
        result = cur.fetchone()
        
        if result==2 :
            print("삭제할 영화 제목이 없습니다.")
        else :
            print("해당 영화 정보를 성공적으로 삭제하였습니다. ")
            
        cur.close()
        
        return result

In [41]:
from math import  *
from statistics import *

class Management:
    def __init__(self):
        self.maria = maria()
        
    def M_select(self): # 출력 함수
        data = self.maria.selectdb() # data는 마리아 클래스의 selectdb에서 갖고옴
        mv = []
        
        if (data):
            for movie in data:
                stp = Movie()
                stp.bunho = movie[0]
                stp.title = movie[1]
                stp.genre = movie[2]
                stp.date = movie[3]
                stp.runtime = movie[4]
                stp.country = movie[5]
                stp.grade = movie[6]
                stp.count = movie[7]
                mv.append(stp)                    
            # studdata=Student((data))
            print(maria.schoolname + "  (개봉예정작 포함)")
            print('%5s %5s %5s %6s %6s %6s %6s %6s' % ('번 호', '제 목', '장 르', '개봉일',
                    '상영시간', '개봉국', '관람 등급', '관람관객 수' ))      
            for stud in mv:
                print(stud)                     
        else :
            print("데이터가 없습니다.")
           
    def M_insert(self):
        stu = Movie()
        stu.inputData()
        self.maria.insertdb(stu.title, stu.genre, stu.date, stu.runtime, stu.country, stu.grade, stu.count)
    
    def M_update(self):
        name = input("검색할 영화를 입력하세요.")
        replace_name = input("수정할 제목을 입력하세요.")
        result = self.maria.updatedb(name, replace_name)
        
    def M_delete(self):       
        title = input("삭제할 영화 제목을 입력하세요.")
        self.maria.deletedb(title)

In [43]:
import sys

class Menu():
    def __init__(self):
        self.manage = Management()
       
    def work(self):
        menu = [ "전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)"]
        manage = Management()
        while 1:
            sel = input(menu)
            if  sel == "1":
                manage.M_select()
                continue
            elif sel == "2":
                manage.M_insert()
                continue       
            elif sel =="3":
                manage.M_update()
                continue
            elif sel =="4":
                manage.M_delete()
            elif sel =="9":
                del(manage)
                sys.exit()
            else :
                print("잘못된 입력입니다. 다시 입력 하세요!")
                continue
           
menu = Menu()
menu.work() 

테이블이 있습니다. 
테이블이 있습니다. 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']2
영화 제목 : 시동
영화 장르 : 드라마
개봉(예정)일: 2019-12-18
상영 시간: 102
개봉국: 한국
관람 등급 : 15세
관람객 수 : 2831736
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
영화 목록표  (개봉예정작 포함)
  번 호   제 목   장 르    개봉일   상영시간    개봉국  관람 등급 관람관객 수
    3 천문: 하늘에 묻는다   드라마 2019-12-26    132     한국      12 1336430 
    4   백두산 드라마, 액션 2019-12-19    128     한국      12 7033339 
    6 겨울왕국2 애니메이션,가족 2019-11-21    103     미국  전체 관람가 13508256 
    7    시동   드라마 2019-12-18    102     한국     15세 2831736 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']2
영화 제목 : 미드웨이
영화 장르 : 액션,드라마
개봉(예정)일: 2019-12-31
상영 시간: 136
개봉국: 미국
관람 등급 : 15세
관람객 수 : 424389
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
영화 목록표  (개봉예정작 포함)
  번 호   제 목   장 르    개봉일   상영시간    개봉국  관람 등급 관람관객 수
    3 천문: 하늘에 묻는다   드라마 2019-12-26    132     한국      12 1336430 
    4   백두산 드라마, 액션 2019-12-19    128     한국      12 7033339 
    6 겨울왕국2 애니메이션,가족 2019-11-21    103     미국  전체 관람가 13508256 
    7    시동   드라마 2019-12-18    102     

SystemExit: 